
Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

Import Dataset

In [14]:
from google.colab import files


uploaded = files.upload()

Saving DataSet.csv to DataSet (1).csv


In [15]:
data = pd.read_csv('DataSet.csv', header=0, sep=',', encoding='Latin1')

Data Preprocessing - Data Cleaning


In [ ]:
data.columns

In [17]:
data.shape

(371528, 20)

In [18]:
data.isna().sum()

dateCrawled                0
name                       0
seller                     0
offerType                  0
price                      0
abtest                     0
vehicleType            37869
yearOfRegistration         0
gearbox                20209
powerPS                    0
model                  20484
kilometer                  0
monthOfRegistration        0
fuelType               33386
brand                      0
notRepairedDamage      72060
dateCreated                0
nrOfPictures               0
postalCode                 0
lastSeen                   0
dtype: int64

In [19]:
data.nunique()

dateCrawled            280500
name                   233531
seller                      2
offerType                   2
price                    5597
abtest                      2
vehicleType                 8
yearOfRegistration        155
gearbox                     2
powerPS                   794
model                     251
kilometer                  13
monthOfRegistration        13
fuelType                    7
brand                      40
notRepairedDamage           2
dateCreated               114
nrOfPictures                1
postalCode               8150
lastSeen               182806
dtype: int64

In [20]:
data.seller.value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

In [21]:
data.drop('offerType', inplace=True,axis=1)

In [22]:
print(data.shape)
data = data[(data.powerPS > 50) & (data.powerPS < 900)]
print(data.shape)

(371528, 19)
(319709, 19)


In [23]:
print(data.shape)
data =  data[(data.yearOfRegistration >1949) & (data.yearOfRegistration<2017)]
print(data.shape)

(319709, 19)
(309171, 19)


In [24]:
data.shape

(309171, 19)

In [25]:
data.nunique()

dateCrawled            244995
name                   189262
seller                      2
price                    5400
abtest                      2
vehicleType                 8
yearOfRegistration         67
gearbox                     2
powerPS                   499
model                     249
kilometer                  13
monthOfRegistration        13
fuelType                    7
brand                      40
notRepairedDamage           2
dateCreated               109
nrOfPictures                1
postalCode               8124
lastSeen               156923
dtype: int64

In [26]:
data.drop(['name', 'abtest', 'dateCrawled', 'nrOfPictures', 'lastSeen', 'postalCode', 'dateCreated'], axis=1, inplace=True)

In [27]:
data.nunique()

seller                    2
price                  5400
vehicleType               8
yearOfRegistration       67
gearbox                   2
powerPS                 499
model                   249
kilometer                13
monthOfRegistration      13
fuelType                  7
brand                    40
notRepairedDamage         2
dtype: int64

In [28]:
data.count()

seller                 309171
price                  309171
vehicleType            297749
yearOfRegistration     309171
gearbox                303873
powerPS                309171
model                  297372
kilometer              309171
monthOfRegistration    309171
fuelType               293284
brand                  309171
notRepairedDamage      265690
dtype: int64

In [29]:
new_data = data.copy()
cols = list(new_data.columns)
cols.remove('brand')
cols
new_data = new_data.drop_duplicates(cols)

In [30]:
cols

['seller',
 'price',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'notRepairedDamage']

In [31]:
new_data.count()

seller                 285145
price                  285145
vehicleType            274091
yearOfRegistration     285145
gearbox                280031
powerPS                285145
model                  273813
kilometer              285145
monthOfRegistration    285145
fuelType               269665
brand                  285145
notRepairedDamage      243965
dtype: int64

In [32]:
new_data.gearbox.replace(('manuell','automatik'),('manual','automatic'), inplace=True)
new_data.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'), inplace=True)
new_data.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),
                            ('small car','convertible','combination','others'), inplace=True)
new_data.notRepairedDamage.replace(('ja','nein'),('Yes','No'), inplace=True)

In [33]:
new_data = new_data[(new_data.price>=100) & (new_data.price<=150000)]
new_data['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_data['fuelType'].fillna(value='not-declared',inplace=True)
new_data['gearbox'].fillna(value='not-declared',inplace=True)
new_data['vehicleType'].fillna(value='not-declared',inplace=True)
new_data['model'].fillna(value='not-declared',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Export Preprocessed Data

In [34]:
new_data.to_csv("autos_preprocessed.csv")

Label Encoding


In [35]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']

mapper = {}
for i in labels:
    mapper[i] = LabelEncoder()
    mapper[i].fit(new_data[i])
    tr = mapper[i].transform(new_data[i])
    np.save(str('classes'+i+'.npy'), mapper[i].classes_)
    new_data.loc[:, i+'_labels'] = pd.Series(tr, index=new_data.index)
    
labeled = new_data[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration']
                  +[x+"_labels" for x in labels]]

print(labeled.columns)

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


Split Data into X & Y

In [36]:
X = labeled.iloc[:,1:].values
Y = labeled.iloc[:,0].values.reshape(-1,1)

Split X & Y into X_train, X_test, Y_train, Y_test

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=3)